In [ ]:
# Initial combined model

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt 

def Ke_Vancomycin(sCr, Age, weight, V=49):
    eCCr = (140 - Age)*weight/(72*sCr)
    Vanco_CL = (60*eCCr)/(1000*1.08)  # l/hr
    Kel = (Vanco_CL/V)  # hr-1
    return Kel

def Ke_Linezolid(CrCL, V):
    CL_linezolid = 5 + (CrCL * 0.4)
    Kel = CL_linezolid / V
    return Kel

def concentration(t, D, V, Ke, tau, start_time, end_time, dosing_times=None):
    conc = np.zeros_like(t)
    if dosing_times is None:
        # If no specific dosing times, create regular dosing schedule
        dosing_times = np.arange(start_time, end_time, tau)
    
    for i, time_point in enumerate(t):
        if start_time <= time_point <= end_time:
            # Calculate contribution from each dose
            for dose_time in dosing_times:
                if time_point >= dose_time:
                    conc[i] += (D / V) * np.exp(-Ke * (time_point - dose_time))
    return conc

#initial_sensitive and initial_resistant are the initial
#populations at the beginning of the simulation
#72 hour period where both bacterial species growth unopposed by an antibiotic
def simulate_drug_dynamics(
    initial_sensitive=2, 
    initial_resistant=5, 
    total_simulation_time=300,
    no_drug_period=72,
    vancomycin_duration=6*24,
    linezolid_duration=8*24,
):
    # Patient Specific Parameters
    sCr = 1.3
    Age = 60
    weight = 70
    V_vancomycin = 49
    V_linezolid = 40

    # Calculate Elimination Rates
    Ke_vanco = Ke_Vancomycin(sCr, Age, weight)
    CrCL = (140 - Age)*weight / (72 * sCr)
    Ke_linez = Ke_Linezolid(CrCL, V_linezolid)

    # Time Array
    time = np.linspace(0, total_simulation_time, total_simulation_time)

    # Vancomycin Dosing
    D_vancomycin = 500  # mg
    tau_vancomycin = 12  # hours
    vanco_start = no_drug_period
    vanco_end = vanco_start + vancomycin_duration

    # Linezolid Dosing
    D_linezolid = 800  # mg
    tau_linezolid = 8  # hours
    linez_start = vanco_end
    linez_end = linez_start + linezolid_duration

    # Create specific dosing times for Linezolid
    linezolid_dosing_times = np.arange(linez_start, linez_end, tau_linezolid)

    # Calculate Drug Concentrations
    conc_vancomycin = concentration(
        time, D_vancomycin, V_vancomycin, Ke_vanco, tau_vancomycin, 
        vanco_start, vanco_end
    )
    conc_linezolid = concentration(
        time, D_linezolid, V_linezolid, Ke_linez, tau_linezolid, 
        linez_start, linez_end, linezolid_dosing_times
    )

    # Combined concentration both vancomycin and linezolid
    conc_combined = conc_vancomycin + conc_linezolid

    # Population Dynamics Parameters
    #Rho values are growth rates
    #Rho term sensitive bacteria
    rho_sensitive = 0.15 #0.5
    #Rho term resistent bacteria
    rho_resistant = 0.3 #0.4
    k = 10e5
    EC_50 = 0.03
    max_drug_effect = 0.8
    #Deltas are death rates
    delta_sensitive = 0.006
    delta_resistant = 0.003

    def population_ode(t, r):
        S, R = r
        conc_t = np.interp(t, time, conc_combined)
        
        drug_inhibition = max_drug_effect * (conc_t / (conc_t + EC_50))
        
        dSdt = rho_sensitive * S * (1 - (S + R) / k) * (1 - drug_inhibition) - delta_sensitive * S
        dRdt = rho_resistant * R * (1 - (S + R) / k) * (1 - 0.5 * drug_inhibition) - delta_resistant * R
        
        return [dSdt, dRdt]

    solution = solve_ivp(
        population_ode, 
        [0, total_simulation_time], 
        [initial_sensitive, initial_resistant],
        dense_output=True
    )

    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 1, 1)
    plt.plot(solution.t, solution.y[0], label='Sensitive Population')
    plt.plot(solution.t, solution.y[1], label='Resistant Population')
    plt.title('Population Dynamics with Drug Effect')
    plt.xlabel('Time (hours)')
    plt.ylabel('Population')
    plt.legend()
    plt.grid(True)

    plt.subplot(2, 1, 2)
    plt.plot(time, conc_vancomycin, label='Vancomycin')
    plt.plot(time, conc_linezolid, label='Linezolid')
    plt.title('Drug Concentration')
    plt.xlabel('Time (hours)')
    plt.ylabel('Concentration')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    #plt.show()

    return solution, time, conc_combined

# Run the simulation
solution, time, conc_combined = simulate_drug_dynamics()

Here’s the equation rendered correctly in Markdown for better clarity:

---

### Logistic Growth Term  

\[
1 - \frac{S + R}{k}
\]

- **\(k\) (Carrying Capacity):** Represents the maximum sustainable population. In this case, \(k = 10^5\).  
- **Behavior:** This term approaches 0 as the total bacterial population (\(S + R\)) approaches \(k\), limiting further growth.  
- **Purpose:** Models competition for resources between bacteria populations.

---

The key is to ensure the equation is enclosed within double dollar signs (`$$`) or single dollar signs (`$`) for inline equations in Markdown environments that support LaTeX rendering, such as Jupyter Notebooks, GitHub, or certain Markdown viewers. Let me know if you'd like to see the entire explanation with all equations formatted properly!

Here’s an improved version of the markdown for better readability and organization:

---

### Explanation of Terms in the Equation

#### Logistic Growth Term  

1 - {S + R}/{k}
 
- **\(k\) (Carrying Capacity):** Represents the maximum sustainable population. In this case, \(k = 10^5\).  
- **Behavior:** This term approaches 0 as the total bacterial population (\(S + R\)) approaches \(k\), limiting further growth.  
- **Purpose:** Models competition for resources between bacteria populations.

#### Total Bacterial Population  

S + R
  
- The sum of the sensitive (\(S\)) and resistant (\(R\)) bacterial populations.

#### Drug Effect Term  

1 - 0.5 {drug_inhibition}
 
- **Drug Inhibition:** Calculated as:

  \[
\text{Drug Inhibition} = \text{Max Drug Effect} \cdot \frac{\text{Concentration at time } t}{\text{Concentration at time } t + EC_{50}}
\]

\[
\text{drug\_inhibition} = \text{max\_drug\_effect} \cdot \frac{\text{conc}_t}{\text{conc}_t + EC_{50}}
\]  
- **Interpretation:**  
  - Drug effect decreases the growth of bacteria based on drug concentration (\(\text{conc}_t\)).  
  - The factor \(0.5\) indicates resistant bacteria are affected only half as much by the drug compared to sensitive bacteria.

#### Natural Death Term  
\[
\delta_\text{resistant}
\]  
- Represents the natural death rate of resistant bacteria.  
- **Independent of Drug Effect:** This term contributes to population decline regardless of drug presence.

---

### Equation Overview  
The equation models the change in the resistant bacterial population (\(R\)) over time, incorporating:  
1. **Growth:** Limited by carrying capacity (\(k\)).  
2. **Drug Inhibition:** Partial inhibition of growth due to drugs.  
3. **Natural Death:** Background mortality independent of drug effects.

---

### Suggested Formatting Enhancements  
- Use equations in LaTeX format to improve clarity.  
- Add headers and bullet points for each major term.  
- Provide brief yet precise explanations for each component.  
- Use bold or italicized text to emphasize key concepts.  

Let me know if you'd like a further reformatting or additional diagrams to clarify these concepts!

In [ ]:
import plotly
print(plotly.__version__)


In [ ]:
import ipywidgets as widgets
print(widgets.__version__)

# Improvements made to initial model

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt 

def Ke_Vancomycin(sCr, Age, weight, V=49):
    eCCr = (140 - Age)*weight/(72*sCr)
    Vanco_CL = (60*eCCr)/(1000*1.08)  # l/hr
    Kel = (Vanco_CL/V)  # hr-1
    return Kel

def Ke_Linezolid(CrCL, V):
    CL_linezolid = 5 + (CrCL * 0.4)
    Kel = CL_linezolid / V
    return Kel

def concentration(t, D, V, Ke, tau, start_time, end_time, dosing_times=None):
    conc = np.zeros_like(t)
    if dosing_times is None:
        dosing_times = np.arange(start_time, end_time, tau)
    
    for i, time_point in enumerate(t):
        if start_time <= time_point <= end_time:
            for dose_time in dosing_times:
                if time_point >= dose_time:
                    conc[i] += (D / V) * np.exp(-Ke * (time_point - dose_time))
    return conc

def simulate_drug_dynamics(
    initial_sensitive=2, 
    initial_resistant=5, 
    no_drug_period=48,        # Reduced from 72
    vancomycin_duration=72,   # Reduced from 144
    linezolid_duration=96,    # Reduced from 192
    total_simulation_time=300
):
    # First, calculate all timing to ensure it fits
    vanco_start = no_drug_period
    vanco_end = vanco_start + vancomycin_duration
    linez_start = vanco_end
    linez_end = linez_start + linezolid_duration
    
    # Verify timing fits within simulation
    if linez_end > total_simulation_time:
        raise ValueError(f"Total treatment time ({linez_end}h) exceeds simulation time ({total_simulation_time}h)")

    # Patient Specific Parameters
    sCr = 1.3
    Age = 60
    weight = 70
    V_vancomycin = 49
    V_linezolid = 40

    # Calculate Elimination Rates
    Ke_vanco = Ke_Vancomycin(sCr, Age, weight)
    CrCL = (140 - Age)*weight / (72 * sCr)
    Ke_linez = Ke_Linezolid(CrCL, V_linezolid)

    # Time Array
    time = np.linspace(0, total_simulation_time, total_simulation_time)

    # Vancomycin Dosing
    D_vancomycin = 500  # mg
    tau_vancomycin = 12  # hours

    # Linezolid Dosing
    D_linezolid = 800  # mg
    tau_linezolid = 8  # hours

    # Create specific dosing times for Linezolid
    #linezolid_dosing_times = np.arange(linez_start, linez_end, tau_linezolid)

    # Calculate Drug Concentrations
    conc_vancomycin = concentration(
        time, D_vancomycin, V_vancomycin, Ke_vanco, tau_vancomycin, 
        vanco_start, vanco_end
    )
    conc_linezolid = concentration(
        time, D_linezolid, V_linezolid, Ke_linez, tau_linezolid, 
        linez_start, linez_end
    )

    # Combined concentration both vancomycin and linezolid
    conc_combined = conc_vancomycin + conc_linezolid

    # Population Dynamics Parameters
    rho_sensitive = 0.3 #increase growth rate of sensitive. making the two rates equal
    rho_resistant = 0.3
    k = 10e5
    EC_50 = 0.03
    max_drug_effect = 0.8
    delta_sensitive = 0.03 #increase death rate sensitive bacteria
    delta_resistant = 0.003

    def population_ode(t, r):
        S, R = r
        conc_t = np.interp(t, time, conc_combined)
        
        drug_inhibition = max_drug_effect * (conc_t / (conc_t + EC_50))
        
        dSdt = rho_sensitive * S * (1 - (S + R) / k) * (1 - drug_inhibition) - delta_sensitive * S
        dRdt = rho_resistant * R * (1 - (S + R) / k) * (1 - 0.8 * drug_inhibition) - delta_resistant * R
        
        return [dSdt, dRdt]

    solution = solve_ivp(
        population_ode, 
        [0, total_simulation_time], 
        [initial_sensitive, initial_resistant],
        dense_output=True,
        t_eval=time
    )

    # Calculate drug inhibition over time
    drug_inhibition = max_drug_effect * (conc_combined / (conc_combined + EC_50))

    # Print key points in the solution
    print("\nKey points in the solution:")
    print(f"Initial conditions:")
    print(f"Sensitive bacteria: {solution.y[0][0]:.2f}")
    print(f"Resistant bacteria: {solution.y[1][0]:.2f}")
    
    print(f"\nAt the start of vancomycin treatment (t={vanco_start}h):")
    idx_vanco = np.where(time >= vanco_start)[0][0]
    print(f"Sensitive bacteria: {solution.y[0][idx_vanco]:.2f}")
    print(f"Resistant bacteria: {solution.y[1][idx_vanco]:.2f}")
    
    print(f"\nAt the end of vancomycin treatment (t={vanco_end}h):")
    idx_vanco_end = np.where(time >= vanco_end)[0][0]
    print(f"Sensitive bacteria: {solution.y[0][idx_vanco_end]:.2f}")
    print(f"Resistant bacteria: {solution.y[1][idx_vanco_end]:.2f}")
    
    print(f"\nAt the end of linezolid treatment (t={linez_end}h):")
    idx_linez_end = np.where(time >= linez_end)[0][0]
    print(f"Sensitive bacteria: {solution.y[0][idx_linez_end]:.2f}")
    print(f"Resistant bacteria: {solution.y[1][idx_linez_end]:.2f}")
    
    print(f"\nFinal values:")
    print(f"Sensitive bacteria: {solution.y[0][-1]:.2f}")
    print(f"Resistant bacteria: {solution.y[1][-1]:.2f}")

    plt.figure(figsize=(15, 15))
    
    # Population dynamics plot
    plt.subplot(3, 1, 1)
    plt.plot(time, solution.y[0], label='Sensitive Population')
    plt.plot(time, solution.y[1], label='Resistant Population')
    plt.axvline(x=vanco_start, color='g', linestyle='--', label='Vancomycin Start')
    plt.axvline(x=vanco_end, color='r', linestyle='--', label='Linezolid Start')
    plt.axvline(x=linez_end, color='k', linestyle='--', label='Treatment End')
    plt.title('Population Dynamics with Drug Effect')
    plt.xlabel('Time (hours)')
    plt.ylabel('Population')
    plt.legend()
    plt.grid(True)

    # Drug concentration plot
    plt.subplot(3, 1, 2)
    plt.plot(time, conc_vancomycin, label='Vancomycin')
    plt.plot(time, conc_linezolid, label='Linezolid')
    plt.title('Drug Concentration')
    plt.xlabel('Time (hours)')
    plt.ylabel('Concentration')
    plt.legend()
    plt.grid(True)

    # Drug inhibition plot
    plt.subplot(3, 1, 3)
    plt.plot(time, drug_inhibition, 'g-', label='Drug Inhibition')
    plt.plot(time, solution.y[0] / np.max(solution.y[0]), 'b--', label='Normalized Sensitive Pop.')
    plt.plot(time, solution.y[1] / np.max(solution.y[1]), 'r--', label='Normalized Resistant Pop.')
    plt.title('Drug Inhibition vs Normalized Population')
    plt.xlabel('Time (hours)')
    plt.ylabel('Relative Value')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    
    return solution, time, conc_combined, drug_inhibition

# Run the simulation
solution, time, conc_combined, drug_inhibition = simulate_drug_dynamics()

## Summary of Improvements made to initial model  
Added calculation of drug_inhibition using the combined drug concentration
The new subplot that shows:
- Drug inhibition over time (green line)
- Normalized sensitive bacterial population (blue dashed line)
- Normalized resistant bacterial population (red dashed line)


Bacterial populations were normalized to better visualize the relationship with drug inhibition

The new plot helps visualize how the drug inhibition affects both bacterial populations over time. In particular:

- The resistant population is less affected by the drug inhibition (as designed in the model)
- The relationship between drug concentration and population decline
- The temporal relationship between peak drug inhibition and population changes


Other changes from initial model include:  


1. Reduced the duration of treatment phases:
   - No drug period: 48h (was 72h)
   - Vancomycin duration: 72h (was 144h)
   - Linezolid duration: 96h (was 192h)
2. Added error checking to ensure total treatment time fits within simulation time
3. Reorganized timing calculations to be more explicit
4. Updated variable names for consistency

